In [1]:
#packages for utilzing the mutagenesis code
import argparse
import re #regular expressions
from Bio import Entrez, SeqIO
Entrez.email = 'sethfrazer.edu'
from skbio import TabularMSA
from skbio import Protein
from skbio.alignment import global_pairwise_align_protein
from Bio.Align import substitution_matrices
import itertools
#packages for utilizing deepBreaks
import os
import subprocess
from deepBreaks.utils import load_obj
from deepBreaks.preprocessing import read_data
import joblib
import pandas as pd
import itertools

In [3]:
def getAcc(accession, check):
    if accession == "manual":

        if check == 0:
            manual = input("Enter Reference Sequence: ")
        else:
            manual = input("Enter Target Sequence: ")

        return(manual)
    
    if accession == "AncBoreotheria":
        return("MSGQEDFYLFENISSVGPWDGPQYHIAPVWAFHLQAAFMGFVFFVGTPLNAIVLVATLRYKKLRQPLNYILVNVSLGGFLFCIFSVFTVFIASCHGYFVFGRHVCALEAFLGSVAGLVTGWSLAFLAFERYIVICKPFGNFRFSSKHALMVVLATWTIGIGVSIPPFFGWSRFIPEGLQCSCGPDWYTVGTKYRSEYYTWFLFIFCFIVPLSLICFSYSQLLRALRAVAAQQQESATTQKAEREVSRMVVVMVGSFCLCYVPYAALAMYMVNNRNHGLDLRLVTIPAFFSKSSCVYNPIIYCFMNKQFRACILETVCGKPMSDESDVSSSAQKTEVSSVSSSQVSPS")
    if accession == "AncEuteleost":
        return("MSGQEDFYLFENISSVSPFEGPQYHLAPKWAFYLQAAFMGFVFFVGTPLNAIVLFVTMKYKKLRQPLNYILVNISLAGFIFVTFSVSQVFVSSTRGYFFLGHTLCALEAAMGSIAGLVTGWSLAVLAFERYVVICKPFGSFKFGSNHALAAVAFTWIIGIGCATPPFFGWSRYIPEGLGCSCGPDWYTKNEEYNSESYTYFLLVTCFIMPLTIIIFSYSQLLGALRAVAAQQAESASTQKAEKEVSRMVIVMVGSFVTCYGPYAITALYFANSTDPNKDYRLVTIPAFFSKSSCVYNPLIYAFMNKQFRACILETVCGKPMSDESDVSSSAQKTEVSSVSSSQVSPS")
    if accession == "AncMammal":
        return("MSGQEDFYLFENISSVGPWDGPQYHIAPAWAFHLQTAFMGFVFFVGTPLNAIVLIATLRYKKLRQPLNYILVNISLAGFIFCIFSVFTVFISSSQGYFVFGRHVCALEAFLGSVAGLVTGWSLAFLAFERYIVICKPFGNFRFNSKHALMVVLATWVIGIGVSIPPFFGWSRYIPEGLQCSCGPDWYTVGTKYRSEYYTWFLFIFCFIVPLSLICFSYSQLLGALRAVAAQQQESATTQKAEREVSRMVVVMVGSFCLCYVPYAALAMYMVNNRNHGLDLRLVTIPAFFSKSSCVYNPIIYCFMNKQFRACILETVCGKPMSDESDVSSSAQKTEVSSVSSSQVSPS")
    if accession == "AncEutheria":
        return("MSGQEDFYLFENISSVGPWDGPQYHIAPVWAFHLQAAFMGFVFFVGTPLNAIVLVATLRYKKLRQPLNYILVNISLGGFLFCIFSVFTVFISSCQGYFVFGRHVCALEAFLGSVAGLVTGWSLAFLAFERYIVICKPFGNFRFSSKHALMVVLATWTIGIGVSIPPFFGWSRFIPEGLQCSCGPDWYTVGTKYRSEYYTWFLFIFCFIVPLSLICFSYSQLLGALRAVAAQQQESATTQKAEREVSRMVVVMVGSFCLCYVPYAALAMYMVNNRNHGLDLRLVTIPAFFSKSSCVYNPIIYCFMNKQFRACILETVCGKPMSDESDVSSSAQKTEVSSVSSSQVSPS")
    if accession == "AncVertebrate":
        return("MSKMSEEEDFYLFGNISSVSPFEGPQYHLAPKWAFYLQAAFMGFVFFVGTPLNAIVLFVTVKYKKLRQPLNYILVNISLGGFLFCIFSVSTVFFSSLRGYFVFGHTVCALEAFLGSVAGLVTGWSLAVLAFERYIVICKPFGNFKFGSKHALMAVVLTWIIGIGCSTPPFFGWSRYIPEGLQCSCGPDWYTVNTEYNSESYTWFLFIFCFIIPLSIITFSYSQLLGALRAVAAQQQESATTQKAEREVSRMVVVMVGSFCVCYVPYAAMALYMVNNRNHGLDLRLVTIPAFFSKSSCVYNPIIYAFMNKQFRACIMETVCGKPMSDDSDVSSSSQKTEVSSVSSSQVSPS")
    if accession == "AncAmphibian":
        return("MSKMSEEEDFYLFKNISNVGPWDGPQYHIAPKWAFYLQAAFMGFVFFVGTPLNAIVLIVTVKYKKLRQPLNYILVNISLGGFLFCIFSVFTVFVSSSQGYFVFGRTVCALEAFLGSVAGLVTGWSLAFLAFERYIVICKPMGNFRFSSKHALMVVVATWIIGIGVSIPPFFGWSRYIPEGLQCSCGPDWYTVGTKYKSEYYTWFLFIFCFIIPLSLICFSYSQLLGALRAVAAQQQESATTQKAEREVSRMVIVMVGSFCLCYVPYAAMAMYMVNNRNHGLDLRLVTIPAFFSKSSCVYNPIIYSFMNKQFRACIMETVCGKPMSDDSSVSSSSQKTEVSSVSSSQVSPS")
    if accession == "lwsancpigmentd":
        return("MAQQWDPQRLAGGQPQDSYEDSTQSSIFTYTNSNSTRGPFEGPNYHIAPRWVYHLTSVWMIFVVIASVFTNGLVLAATMKFKKLRHPLNWILVNLAIADLAETVIASTISVVNQIYGYFVLGHPMCVVEGYTVSLCGITGLWSLAIISWERWLVVCKPFGNVRFDAKLAIVGIAFSWIWSAVWTAPPIFGWSRYWPHGLKTSCGPDVFSGSSYPGVQSYMIVLMVTCCIIPLSIIILCYLQVWLAIRAVAKQQKESESTQKAEKEVTRMVVVMIFAYCLCWGPYTFFACFAAANPGYAFHPLVAALPAYFAKSATIYNPIIYVFMNRQFRNCILQLFGKKVDDSSELSSASKTEVSSVSSVSPA")

    try:
        handle = Entrez.efetch(db="nucleotide", id=accession, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "gb")
        handle.close()
        for i,feature in enumerate(record.features):
            if feature.type=='CDS':
                aa = feature.qualifiers['translation'][0]
    except:
        manual = input(f"Accession# {accession} - Not Recognized by NCBI\nPlease Enter Target Sequence to Continue: ")
        return(manual)
    return(aa)

In [4]:
def getCombos(mut_list, wt_seq, mut_acc_file):
    wt_seq = f'{wt_seq}_'
    x = 1
    mut_combos = []
    while x in range(len(mut_list)+1):
        combinations = list(itertools.combinations(mut_list, x))

        for combo in combinations:
            acc = wt_seq

            check = 1
            for i in combo:
                if check == x:    
                    acc+=i
                else:
                    acc = f'{acc}{i},'
                check+=1
            print(acc)

            mut_combos.append(acc)
        x +=1
    #print(mut_combos)
    #print(len(mut_combos))

    with open(mut_acc_file, 'w') as f:
        for mut_acc in mut_combos:
            f.write(f'{mut_acc}\n')
    
    return(mut_combos)

In [5]:
def getMutAcc_fromCSV(mut_file, acc_name):
    # Replace 'your_file.csv' with the path to your CSV file    
    df = pd.read_csv(mut_file)

    #  Access and print the column headers
    column_headers = list(df.columns)
    print(column_headers)

    base_acc = acc_name
    list_of_accs = []

    for row in df.itertuples():
        current_acc = base_acc
        check = 0
        # Iterate through each column value
        for i in range(1, len(row)):
            if row[i] == 1:
                if check == 0:
                    current_acc += '_' + column_headers[i-1]
                    check+=1
                else:
                    current_acc += ',' + column_headers[i-1]
        
        # Append the constructed string to the list and reset current_acc
        if current_acc != base_acc:
            list_of_accs.append(current_acc)
            
        current_acc = base_acc
    print(list_of_accs)
    return(list_of_accs)

In [11]:
def getMut(mut_combos,raccession,output_file, out_format='fasta', allow_wt = True):
    switch = 0
    bovine = Protein(getAcc(raccession, switch))
    switch+=1

    try:
        mut_combos = open(mut_combos).readlines()
    except:
        pass


    #Fetch sequences to manipulate and align
    test = 0
    if out_format != 'fasta':
        with open(output_file, 'w') as f:
            f.write(f'Accessions\tMutant_Seqs\n')
    else:
        with open(output_file, 'w') as f:
            f.write('')
    for lines in mut_combos:     
    
        if len(lines.split("_")[0]) == 2:
            try:
                accession = f'{lines.split("_")[0]}_{lines.split("_")[1]}'
                mutation = lines.split('_')[2]
            except:
                accession = f'{lines.split("_")[0]}_{lines.split("_")[1]}'
                mutation = []

        elif '_' not in lines:
            accession = lines
            mutation = []
            wt = Protein(getAcc(accession, switch))

        else:
            accession = f'{lines.split("_")[0]}'
            mutation = lines.split('_')[1]

        accession = accession.replace('\n','')
        wt = Protein(getAcc(accession, switch))

        substitution_matrix = substitution_matrices.load("BLOSUM45")

        ##Simple example for testing
        #wt=Protein("ABCDEF")
        #bovine=Protein("ABCDEF")

        alignment, score, start_end_positions = global_pairwise_align_protein(bovine, wt, substitution_matrix=substitution_matrix, gap_open_penalty=20)
        dic = alignment.to_dict()
        aligned_bovine = dic[0]
        aligned_wt = dic[1]

        if test == 0:
            print(f'This is the aligned bovine:\n{aligned_bovine}')
            print(f'This is {accession} aligned to bovine:\n{aligned_wt}')
        
        #Check if any mutation is entered
        full_mutation = mutation
        if len(mutation) == 0:
            if allow_wt == True:
                print("You've requested a WT Sequence")
                if out_format == 'fasta':
                    with open(output_file, 'a') as f:
                        f.write(f'>{accession}\n{wt}\n')

                else:
                    with open(output_file, 'a') as f:
                        f.write(f'{accession}\t{wt}\n')
            else:
                raise Exception("ERROR: Expecting mutation name in the format of XaY where X=old amino acid a=number Y=new amino acid, but received WT sequence")
        else: 
            #Check if single or multiple mutations entered
            if "," in mutation :
                mutations = mutation.split(",")
            else :
                #declare empty list and append single mutation so later can be used as list if only one mutation
                mutations = []
                mutations.append(mutation)

            mutated = aligned_wt #need to copy original sequence and accumulate mutations
            for mutation in mutations:

                #check that mutation is formatted properly
                MutationRegex = re.compile(r'([A-Z])(\d+)([A-Z])')
                if re.match(MutationRegex, mutation):
                    mu = MutationRegex.search(mutation)
                    old = mu.group(1)
                    mutsite = int(mu.group(2))
                    new = mu.group(3)
                elif 'del' in mutation:
                    MutationRegex = re.compile(r'([A-Z])(\d+)([a-z]+)')
                    mu = MutationRegex.search(mutation)
                    old = mu.group(1)
                    mutsite = int(mu.group(2))
                    new = mu.group(3)
                elif 'ins' in mutation:
                    MutationRegex = re.compile(r'([a-z]+)(\d+)([A-Z])')
                    mu = MutationRegex.search(mutation)
                    old = mu.group(1)
                    mutsite = int(mu.group(2))
                    new = mu.group(3)
                else :
                    raise Exception("ERROR: Expecting mutation name in the format of XaY where X=old amino acid a=number Y=new amino acid")

                #substract one to account for index starting at zero
                #print(mutsite)
                mutsite = mutsite-1
                #print(mutsite)

                #Count gaps to not count them when finding original site to change
                gaps = aligned_bovine[:mutsite].count('-')
                #print(gaps)


                if new == 'del':
                    new = ''
                if old == 'ins':
                    mutated = str(mutated[:mutsite]) + new + str(mutated[mutsite:])
                else:
                    #match = str(mutated[mutsite+gaps])
                    match = str(mutated[mutsite])

                    #print(match)
                    if match != old :
                        try: 
                            match = str(mutated[mutsite+gaps])
                            if match != old: 
                                    raise Exception()
                            else:
                                mutated = str(mutated[:mutsite+gaps]) + new + str(mutated[mutsite+gaps+1:]) 
                        except:
                            print(f'The AA at site {mutsite+1} is supposed to be {old} but appears to be {match}')
                            raise Exception("ERROR: The mutation label does not match the sequence")
                    else: 
                        mutated = str(mutated[:mutsite]) + new + str(mutated[mutsite+1:]) 

            mutated = mutated.replace('-','')
            mutated.replace('\n','')
            newacc = accession + "_" + full_mutation
            newacc = newacc.replace('\n','')

            if out_format == 'fasta':
                with open(output_file, 'a') as f:
                    f.write(f'>{newacc}\n{mutated}\n')
                    print(newacc, end='\t\t')
                    print(mutated)
            else:
                with open(output_file, 'a') as f:
                    f.write(f'{newacc}\t{mutated}\n')
                    print(newacc, end='\t\t')
                    print(mutated)

    return(newacc , mutated)

In [13]:
def getMutPred(report_dir, seq_add, seqFileName, metaDataFileName, mafft_exe, mt, seq_type, gap_threshold, top_per_mod, prediction_report_dir):

    load_top_mod = load_obj(top_per_mod)

    mut_ali = seqFileName[2:].split("/")[1]
    mut_ali = f'{mut_ali.split(".")[0]}_mutant_aligned.fasta'
    print(mut_ali)
    cmd = [mafft_exe, '--add', seq_add, '--keeplength', seqFileName , '>', f'{report_dir}/{mut_ali}' ]
    aligner = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out_put = aligner.communicate()[0].decode('utf8')

    #print(out_put)
    mut_test = read_data( f'./{report_dir}{mut_ali}', seq_type = seq_type, is_main=True, gap_threshold=gap_threshold)
    ref_copy = read_data( seqFileName, seq_type = seq_type, is_main=True, gap_threshold=gap_threshold)
    last_seq = ref_copy.shape[0]
    print(ref_copy.shape)
    #print(last_seq)
    mut_test = mut_test.iloc[last_seq:].copy()
    print(mut_test)

    predictions = load_top_mod.predict(mut_test)
    print(predictions)
    mut_fin = pd.DataFrame(index=mut_test.index)
    mut_fin['Prediction'] = predictions
    mut_fin.to_csv(path_or_buf= prediction_report_dir,mode="w")

    return('Model Predictions Complete!')

In [23]:
#mut_acc_list = getMutAcc_fromCSV('human_rho_mutatnts.csv', 'AncBoreotheria')
raccession = 'AH005298.2'
input_file = 'seq_acc.txt'
output_file = 'full_viz_seqs.tsv'
getMut(input_file,raccession,output_file,out_format='tsv')

This is the aligned bovine:
MAQQWSLQRLAGRHPQDSYEDSTQSSIFTYTNSNSTRGPFEGPNYHIAPRWVYHLTSVWMIFVVTASVFTNGLVLAATMKFKKLRHPLNWILVNLAVADLAETVIASTISIVNQVSGYFVLGHPMCVLEGYTVSLCGITGLWSLAIISWERWLVVCKPFGNVRFDAKLAIVGIAFSWIWSAVWTAPPIFGWSRYWPHGLKTSCGPDVFSGSSYPGVQSYMIVLMVTCCIIPLAIIMLCYLQVWLAIRAVAKQQKESESTQKAEKEVTRMVVVMIFAYCVCWGPYTFFACFAAANPGYAFHPLMAALPAYFAKSATIYNPVIYVFMNRQFRNCILQLFGKKVDDGSELSSASKTEVSSVSSVSPA
This is AF011389.1 aligned to bovine:
-----MAQRLTGEQTLDHYEDSTHASIFTYTNSNSTKGPFEGPNYHIAPRWVYHLTSTWMILVVVASVFTNGLVLAATMRFKKLRHPLNWILVNLAVADLAETIIASTISVVNQIYGYFVLGHPLCVIEGYIVSLCGITGLWSLAIISWERWLVVCKPFGNVRFDAKLATVGIVFSWVWAAIWTAPPIFGWSRYWPYGLKTSCGPDVFSGTSYPGVQSYMMVLMVTCCIFPLSIIVLCYLQVWLAIRAVAKQQKESESTQKAEKEVTRMVVVMVFAYCLCWGPYTFFACFATAHPGYAFHPLVASLPSYFAKSATIYNPIIYVFMNRQFRNCILHLFGKKVDDSSELSSTSKTEVSSVSSVSPA
AF011389.1_S308A		MAQRLTGEQTLDHYEDSTHASIFTYTNSNSTKGPFEGPNYHIAPRWVYHLTSTWMILVVVASVFTNGLVLAATMRFKKLRHPLNWILVNLAVADLAETIIASTISVVNQIYGYFVLGHPLCVIEGYIVSLCGITGLWSLAIISWERWLVVCKPFGNVRFDAKLATVGIVFSWVWAAIWTAPPIFGW

('AH005298.2_H197Y,A308S',
 'MAQQWSLQRLAGRHPQDSYEDSTQSSIFTYTNSNSTRGPFEGPNYHIAPRWVYHLTSVWMIFVVTASVFTNGLVLAATMKFKKLRHPLNWILVNLAVADLAETVIASTISIVNQVSGYFVLGHPMCVLEGYTVSLCGITGLWSLAIISWERWLVVCKPFGNVRFDAKLAIVGIAFSWIWSAVWTAPPIFGWSRYWPYGLKTSCGPDVFSGSSYPGVQSYMIVLMVTCCIIPLAIIMLCYLQVWLAIRAVAKQQKESESTQKAEKEVTRMVVVMIFAYCVCWGPYTFFACFAAANPGYAFHPLMAALPSYFAKSATIYNPVIYVFMNRQFRNCILQLFGKKVDDGSELSSASKTEVSSVSSVSPA')

In [14]:
wds_mut_list = ['A132S', 'N151S', 'G174D', 'P180R', 'F228L', 'F287L']
wt = 'NM_001280659'
mut_acc_file = 'dolphin_muts_csts.txt'
raccession = 'NM_001014890'
mutant_seq_file = 'dolphin_muts_csts.fasta'
out_format = 'fasta'
combos = getCombos(wds_mut_list, wt, mut_acc_file)
getMut(combos,raccession,mutant_seq_file, out_format)

NM_001280659_A132S
NM_001280659_N151S
NM_001280659_G174D
NM_001280659_P180R
NM_001280659_F228L
NM_001280659_F287L
NM_001280659_A132S,N151S
NM_001280659_A132S,G174D
NM_001280659_A132S,P180R
NM_001280659_A132S,F228L
NM_001280659_A132S,F287L
NM_001280659_N151S,G174D
NM_001280659_N151S,P180R
NM_001280659_N151S,F228L
NM_001280659_N151S,F287L
NM_001280659_G174D,P180R
NM_001280659_G174D,F228L
NM_001280659_G174D,F287L
NM_001280659_P180R,F228L
NM_001280659_P180R,F287L
NM_001280659_F228L,F287L
NM_001280659_A132S,N151S,G174D
NM_001280659_A132S,N151S,P180R
NM_001280659_A132S,N151S,F228L
NM_001280659_A132S,N151S,F287L
NM_001280659_A132S,G174D,P180R
NM_001280659_A132S,G174D,F228L
NM_001280659_A132S,G174D,F287L
NM_001280659_A132S,P180R,F228L
NM_001280659_A132S,P180R,F287L
NM_001280659_A132S,F228L,F287L
NM_001280659_N151S,G174D,P180R
NM_001280659_N151S,G174D,F228L
NM_001280659_N151S,G174D,F287L
NM_001280659_N151S,P180R,F228L
NM_001280659_N151S,P180R,F287L
NM_001280659_N151S,F228L,F287L
NM_001280659_G1

('NM_001280659_A132S,N151S,G174D,P180R,F228L,F287L',
 'MNGTEGLNFYVPFSNKTGVVRSPFEYPQYYLAEPWQFSVLAAYMFLLIVLGFPINFLTLYVTVQHKKLRTPLNYILLNLAVANLFMVFGGFTTTLYTSLHAYFVFGPTGCNLEGFFATLGGEIALWSLVVLSIERYVVVCKPMSNFRFGESHAIMGLALTWIMAMACAAAPLVDWSRYIREGMQCSCGIDYYTSRQEVNNESFVIYMFVVHFTIPLVIIFFCYGQLVLTVKEAAAQQQESATTQKAEKEVTRMVIIMVVAFLICWVPYASVAFYIFTHQGSDFGPILMTIPSFFAKSSSIYNPVIYIMMNKQFRNCMLTTLCCGRNPLGDDEASTTASKTETSQVAPA')

In [ ]:
wds_ni_mut_list = ['S298A', 'M44I', 'C162I', 'P215C', 'M257E', 'R314F']
wt = 'AB087809'
mut_acc_file = 'dan_rer_ni_mut_acc.txt'
raccession = 'NM_001014890'
mutant_seq_file = 'dan_rer_sws_ni_mut_seqs.fasta'
combos = getCombos(wds_ni_mut_list, wt, mut_acc_file)
getMut(combos,raccession,mutant_seq_file)

In [ ]:
nmoc_mut_list = ['F46L', 'E181H', 'T116F', 'C264A', 'A269T', 'P303S']
wt = 'AB087809'
mut_acc_file = 'dan_rer_nmoc_mut_acc.txt'
raccession = 'NM_001014890'
mutant_seq_file = 'dan_rer_sws_nmoc_mut_seqs.fasta'
combos = getCombos(nmoc_mut_list, wt, mut_acc_file)
getMut(combos,raccession,mutant_seq_file)

In [ ]:
#path to sequences we want to add to an existing alignment [the one we used to train the model] in FASTA format
#seq_add = mutant_seq_file
seq_add = 'dan_rer_sws_mut_seqs.fasta'
#path to the primary alignment used for training the model - if trying access a specific file later
report_dir = './gapt_60_wds_aligned_db_fmt_Lambda_Max_2023-08-10_14-38-42/'
seqFileName = f'{report_dir}wds_aligned_db_fmt.fasta' 
# path to the corresponding metadata file
metaDataFileName = f'{report_dir}wds_meta.tsv' 
#path to the mafft.bat file
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
# name of the phenotype
mt = 'Lambda_Max'
# type of the sequences
seq_type = 'aa'
#if the proportion of gaps at position 'x' across the whole dataset are > threshold, drop this position
gap_threshold = 0.6
#path to the top performing model from model training
#can use method below or enter path manually if coming back to notebook with no variables intialized
top_per_mod = report_dir + 'lgbm.pkl'
#Desired path for the predictions output csv...
prediction_report_dir = f'{report_dir}wds_sws_mutant_predictions.csv'

getMutPred(report_dir,seq_add,seqFileName,metaDataFileName,mafft_exe,mt,seq_type,gap_threshold,top_per_mod,prediction_report_dir)

In [ ]:
#path to sequences we want to add to an existing alignment [the one we used to train the model] in FASTA format
#seq_add = mutant_seq_file
seq_add = 'dan_rer_sws_ni_mut_seqs.fasta'
#path to the primary alignment used for training the model - if trying access a specific file later
report_dir = './gapt_60_wds_ni_aligned_db_fmt_Lambda_Max_2023-08-10_12-01-59/'
seqFileName = f'{report_dir}wds_ni_aligned_db_fmt.fasta' 
# path to the corresponding metadata file
metaDataFileName = f'{report_dir}wds_ni_meta.tsv' 
#path to the mafft.bat file
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
# name of the phenotype
mt = 'Lambda_Max'
# type of the sequences
seq_type = 'aa'
#if the proportion of gaps at position 'x' across the whole dataset are > threshold, drop this position
gap_threshold = 0.61
#path to the top performing model from model training
#can use method below or enter path manually if coming back to notebook with no variables intialized
top_per_mod = report_dir + 'lgbm.pkl'
#Desired path for the predictions output csv...
prediction_report_dir = f'{report_dir}wds_ni_sws_mutant_predictions.csv'

getMutPred(report_dir,seq_add,seqFileName,metaDataFileName,mafft_exe,mt,seq_type,gap_threshold,top_per_mod,prediction_report_dir)

In [ ]:
#path to sequences we want to add to an existing alignment [the one we used to train the model] in FASTA format
#seq_add = mutant_seq_file
seq_add = 'dan_rer_sws_nmoc_mut_seqs.fasta'
#path to the primary alignment used for training the model - if trying access a specific file later
report_dir = './gapt_60_nmoc_mut_added_aligned_db_fmt_Lambda_Max_2023-08-10_11-37-58/'
seqFileName = f'{report_dir}nmoc_mut_added_aligned_db_fmt.fasta' 
# path to the corresponding metadata file
metaDataFileName = f'{report_dir}nmoc_meta.tsv' 
#path to the mafft.bat file
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
# name of the phenotype
mt = 'Lambda_Max'
# type of the sequences
seq_type = 'aa'
#if the proportion of gaps at position 'x' across the whole dataset are > threshold, drop this position
gap_threshold = 0.6
#path to the top performing model from model training
#can use method below or enter path manually if coming back to notebook with no variables intialized
top_per_mod = report_dir + 'BayesianRidge.pkl'
#Desired path for the predictions output csv...
prediction_report_dir = f'{report_dir}nmoc_sws_mutant_predictions.csv'

getMutPred(report_dir,seq_add,seqFileName,metaDataFileName,mafft_exe,mt,seq_type,gap_threshold,top_per_mod,prediction_report_dir)

In [ ]:
wds_mut_list = ['V162T', 'K67T', 'Y178F', 'L72Q', 'E247M', 'R311I']
wt = 'U08131'
mut_acc_file = 'anol_car_mut_acc.txt'
raccession = 'NM_001014890'
mutant_seq_file = 'anol_car_lws_mut_seqs.fasta'
combos = getCombos(wds_mut_list, wt, mut_acc_file)
getMut(combos,raccession,mutant_seq_file)

In [ ]:
wds_ni_mut_list = ['A298S', 'M44I', 'V162I', 'P215C', 'M257E', 'R314F']
wt = 'U08131'
mut_acc_file = 'anol_car_mut_acc.txt'
raccession = 'NM_001014890'
mutant_seq_file = 'anol_car_lws_ni_mut_seqs.fasta'
combos = getCombos(wds_ni_mut_list, wt, mut_acc_file)
getMut(combos,raccession,mutant_seq_file)

In [ ]:
nmoc_mut_list = ['F46L', 'H181E', 'T116F', 'C264A', 'T269A', 'P303S']
wt = 'U08131'
mut_acc_file = 'anol_car_mut_acc.txt'
raccession = 'NM_001014890'
mutant_seq_file = 'anol_car_lws_nmoc_mut_seqs.fasta'
combos = getCombos(nmoc_mut_list, wt, mut_acc_file)
getMut(combos,raccession,mutant_seq_file)

In [ ]:
#path to sequences we want to add to an existing alignment [the one we used to train the model] in FASTA format
seq_add = 'anol_car_lws_nmoc_mut_seqs.fasta'
#path to the primary alignment used for training the model - if trying access a specific file later
report_dir = './gapt_60_wds_aligned_db_fmt_Lambda_Max_2023-08-10_14-38-42/'
seqFileName = f'{report_dir}wds_aligned_db_fmt.fasta' 
# path to the corresponding metadata file
metaDataFileName = f'{report_dir}wds_meta.tsv' 
#path to the mafft.bat file
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
# name of the phenotype
mt = 'Lambda_Max'
# type of the sequences
seq_type = 'aa'
#if the proportion of gaps at position 'x' across the whole dataset are > threshold, drop this position
gap_threshold = 0.6
#path to the top performing model from model training
#can use method below or enter path manually if coming back to notebook with no variables intialized
top_per_mod = report_dir + 'lgbm.pkl'
#Desired path for the predictions output csv...
prediction_report_dir = f'{report_dir}nmoc_lws_mutant_predictions.csv'

getMutPred(report_dir,seq_add,seqFileName,metaDataFileName,mafft_exe,mt,seq_type,gap_threshold,top_per_mod,prediction_report_dir)

In [ ]:
#path to sequences we want to add to an existing alignment [the one we used to train the model] in FASTA format
#seq_add = mutant_seq_file
seq_add = 'anol_car_lws_ni_mut_seqs.fasta'
#path to the primary alignment used for training the model - if trying access a specific file later
report_dir = './gapt_60_wds_ni_aligned_db_fmt_Lambda_Max_2023-08-10_12-01-59/'
seqFileName = f'{report_dir}wds_ni_aligned_db_fmt.fasta' 
# path to the corresponding metadata file
metaDataFileName = f'{report_dir}wds_ni_meta.tsv' 
#path to the mafft.bat file
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
# name of the phenotype
mt = 'Lambda_Max'
# type of the sequences
seq_type = 'aa'
#if the proportion of gaps at position 'x' across the whole dataset are > threshold, drop this position
gap_threshold = 0.61
#path to the top performing model from model training
#can use method below or enter path manually if coming back to notebook with no variables intialized
top_per_mod = report_dir + 'lgbm.pkl'
#Desired path for the predictions output csv...
prediction_report_dir = f'{report_dir}wds_ni_lws_mutant_predictions.csv'

getMutPred(report_dir,seq_add,seqFileName,metaDataFileName,mafft_exe,mt,seq_type,gap_threshold,top_per_mod,prediction_report_dir)

In [ ]:
#path to sequences we want to add to an existing alignment [the one we used to train the model] in FASTA format
#seq_add = mutant_seq_file
seq_add = 'anol_car_lws_nmoc_mut_seqs.fasta'
#path to the primary alignment used for training the model - if trying access a specific file later
report_dir = './gapt_60_nmoc_mut_added_aligned_db_fmt_Lambda_Max_2023-08-10_11-37-58/'
seqFileName = f'{report_dir}nmoc_mut_added_aligned_db_fmt.fasta' 
# path to the corresponding metadata file
metaDataFileName = f'{report_dir}nmoc_meta.tsv' 
#path to the mafft.bat file
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
# name of the phenotype
mt = 'Lambda_Max'
# type of the sequences
seq_type = 'aa'
#if the proportion of gaps at position 'x' across the whole dataset are > threshold, drop this position
gap_threshold = 0.6
#path to the top performing model from model training
#can use method below or enter path manually if coming back to notebook with no variables intialized
top_per_mod = report_dir + 'BayesianRidge.pkl'
#Desired path for the predictions output csv...
prediction_report_dir = f'{report_dir}nmoc_lws_mutant_predictions.csv'

getMutPred(report_dir,seq_add,seqFileName,metaDataFileName,mafft_exe,mt,seq_type,gap_threshold,top_per_mod,prediction_report_dir)

PREDICTIONS FOR UNTESTED VERT OPSINS:

In [ ]:
#path to sequences we want to add to an existing alignment [the one we used to train the model] in FASTA format
seq_add = 'untested_vert_seqs.fasta'
#path to the primary alignment used for training the model - if trying access a specific file later
report_dir = './gapt_60_wds_aligned_db_fmt_Lambda_Max_2023-08-10_14-38-42/'
seqFileName = f'{report_dir}wds_aligned_db_fmt.fasta' 
# path to the corresponding metadata file
metaDataFileName = f'{report_dir}wds_meta.tsv' 
#path to the mafft.bat file
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
# name of the phenotype
mt = 'Lambda_Max'
# type of the sequences
seq_type = 'aa'
#if the proportion of gaps at position 'x' across the whole dataset are > threshold, drop this position
gap_threshold = 0.6
#path to the top performing model from model training
#can use method below or enter path manually if coming back to notebook with no variables intialized
top_per_mod = report_dir + 'lgbm.pkl'
#Desired path for the predictions output csv...
prediction_report_dir = f'{report_dir}wds_untested_vert_seqs_predictions.csv'

getMutPred(report_dir,seq_add,seqFileName,metaDataFileName,mafft_exe,mt,seq_type,gap_threshold,top_per_mod,prediction_report_dir)

In [ ]:
#path to sequences we want to add to an existing alignment [the one we used to train the model] in FASTA format
#seq_add = mutant_seq_file
seq_add = 'untested_vert_seqs.fasta'
#path to the primary alignment used for training the model - if trying access a specific file later
report_dir = './gapt_60_wds_ni_aligned_db_fmt_Lambda_Max_2023-08-10_12-01-59/'
seqFileName = f'{report_dir}wds_ni_aligned_db_fmt.fasta' 
# path to the corresponding metadata file
metaDataFileName = f'{report_dir}wds_ni_meta.tsv' 
#path to the mafft.bat file
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
# name of the phenotype
mt = 'Lambda_Max'
# type of the sequences
seq_type = 'aa'
#if the proportion of gaps at position 'x' across the whole dataset are > threshold, drop this position
gap_threshold = 0.60
#path to the top performing model from model training
#can use method below or enter path manually if coming back to notebook with no variables intialized
top_per_mod = report_dir + 'lgbm.pkl'
#Desired path for the predictions output csv...
prediction_report_dir = f'{report_dir}wds_ni_untested_verts_predictions.csv'

getMutPred(report_dir,seq_add,seqFileName,metaDataFileName,mafft_exe,mt,seq_type,gap_threshold,top_per_mod,prediction_report_dir)

In [ ]:
#path to sequences we want to add to an existing alignment [the one we used to train the model] in FASTA format
#seq_add = mutant_seq_file
seq_add = 'untested_vert_seqs.fasta'
#path to the primary alignment used for training the model - if trying access a specific file later
report_dir = './gapt_60_nmoc_mut_added_aligned_db_fmt_Lambda_Max_2023-08-10_11-37-58/'
seqFileName = f'{report_dir}nmoc_mut_added_aligned_db_fmt.fasta' 
# path to the corresponding metadata file
metaDataFileName = f'{report_dir}nmoc_meta.tsv' 
#path to the mafft.bat file
mafft_exe = 'C:/Users/safra/mafft-win/mafft.bat'
# name of the phenotype
mt = 'Lambda_Max'
# type of the sequences
seq_type = 'aa'
#if the proportion of gaps at position 'x' across the whole dataset are > threshold, drop this position
gap_threshold = 0.6
#path to the top performing model from model training
#can use method below or enter path manually if coming back to notebook with no variables intialized
top_per_mod = report_dir + 'BayesianRidge.pkl'
#Desired path for the predictions output csv...
prediction_report_dir = f'{report_dir}nmoc_untested_verts_predictions.csv'

getMutPred(report_dir,seq_add,seqFileName,metaDataFileName,mafft_exe,mt,seq_type,gap_threshold,top_per_mod,prediction_report_dir)